In [1]:
import pandas as pd
import numpy as np
import keras

Using TensorFlow backend.


# 1. Data Preparation and Cleaning

In [2]:
concrete_data = pd.read_csv('https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0101EN/labs/data/concrete_data.csv')
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


Check the ammount of data and the number of columns
<em>(row,columns)</em>

In [3]:
concrete_data.shape 

(1030, 9)

Check if there are any missing values

In [4]:
concrete_data.describe()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


Check if there are any null rows

In [5]:
concrete_data.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

## Split the data into predictors and Target
For the predictors you'll have to be selective on which columns to use and which ones to leave for target


In [37]:
concrete_data_columns = concrete_data.columns
predictors = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']] #All the columns except strength
predictors_columns = predictors.columns
predictors = predictors[predictors_columns[predictors_columns != 'Age']] #All the columns except age
target = concrete_data['Strength'] 

In [38]:
predictors.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5


In [8]:
target.head()

0    79.99
1    61.89
2    40.27
3    41.05
4    44.30
Name: Strength, dtype: float64

In [39]:
predictors_norm = (predictors - predictors.mean())/predictors.std()
predictors_norm.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569


The number of columns to prepare for the first hidden layer

In [43]:
n_cols = predictors_norm.shape[1]

# 2. Train the Model

In [44]:
from keras.models import Sequential
from keras.layers import Dense

### Prepare a function used to prepare a model

The function below creates a model with two hidden layers having 50 neurons each.

In [45]:
def regression_model():
    #Create the model
    model = Sequential()
    model.add(Dense(50, activation = 'relu', input_shape = (n_cols,)))
    model.add(Dense(50, activation = 'relu'))
    model.add(Dense(1))
    #Compile the model
    model.compile(optimizer = 'adam',loss='mean_squared_error')
    return model

In [46]:
model = regression_model()

# 3. Train the Model

In [54]:
xx = model.fit(predictors_norm, target, validation_split = 0.3, epochs = 100, verbose = 2)

Train on 721 samples, validate on 309 samples
Epoch 1/100
 - 0s - loss: 151.6591 - val_loss: 155.6167
Epoch 2/100
 - 0s - loss: 151.5303 - val_loss: 159.0440
Epoch 3/100
 - 0s - loss: 151.1539 - val_loss: 159.3351
Epoch 4/100
 - 0s - loss: 150.9955 - val_loss: 154.2132
Epoch 5/100
 - 0s - loss: 150.9907 - val_loss: 158.3036
Epoch 6/100
 - 0s - loss: 150.6952 - val_loss: 157.4052
Epoch 7/100
 - 0s - loss: 150.5741 - val_loss: 152.1793
Epoch 8/100
 - 0s - loss: 150.3525 - val_loss: 155.9857
Epoch 9/100
 - 0s - loss: 150.3787 - val_loss: 155.0797
Epoch 10/100
 - 0s - loss: 150.4002 - val_loss: 154.5817
Epoch 11/100
 - 0s - loss: 150.2174 - val_loss: 158.0890
Epoch 12/100
 - 0s - loss: 150.0240 - val_loss: 160.6074
Epoch 13/100
 - 0s - loss: 149.2604 - val_loss: 157.9198
Epoch 14/100
 - 0s - loss: 149.7758 - val_loss: 155.6406
Epoch 15/100
 - 0s - loss: 149.3576 - val_loss: 160.9311
Epoch 16/100
 - 0s - loss: 150.0328 - val_loss: 156.3158
Epoch 17/100
 - 0s - loss: 149.5289 - val_loss: 158

In [21]:
y_pred = model.predict()

TypeError: predict() missing 1 required positional argument: 'x'

In [49]:
a = []
a.append(19)
a.append(129)

In [50]:
a

[19, 129]

In [51]:
np.mean(a)

74.0

In [52]:
np.std(a)

55.0

In [53]:
np.std?